# Preprocessing and clustering

In [ ]:
# Core scverse libraries
from __future__ import annotations

import anndata as ad

# Data retrieval
import pooch
import scanpy as sc

sc.settings.set_figure_params(dpi=50, facecolor="white")

# 读取数据 - read_10x_h5()

In [ ]:
s1d1 = sc.read_10x_h5('scverse_tutorials/s1d1_filtered_feature_bc_matrix.h5')
s1d3 = sc.read_10x_h5('scverse_tutorials/s1d3_filtered_feature_bc_matrix.h5')
s1d1.var_names_make_unique()
s1d3.var_names_make_unique()

# 合并样本 - ad.concat()

In [ ]:
# 合并样本
adatas = {}
adatas['s1d1'] = s1d1
adatas['s1d3'] = s1d3

adata = ad.concat(adatas, label="sample")
adata.obs_names_make_unique()

In [ ]:
s1d1

In [ ]:
s1d3

# 质控 - calculate_qc_metrics()

可以将特定的基因群体传递到 calculate_qc_metrics（）， 以计算这些群体的计数比例。线粒体基因、核糖体基因和血红蛋白基因由以下列出的不同前缀定义。

In [ ]:
# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

In [ ]:
adata.obs

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=False)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

有时看似质量控制不佳的指标可能由真实生物学驱动，因此我们建议从非常宽松的过滤策略开始，稍后再重新评估。
因此，我们现在只过滤表达少于 100 个基因和检测不到 3 个细胞的基因。

此外，需要注意的是，对于多批次的数据集，质量控制应针对每个样本单独进行，因为不同批次的质量控制阈值可能差异较大。

In [ ]:
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=3)

# 双胞检测 - scrublet

In [ ]:
sc.pp.scrublet(adata, batch_key="sample")

In [ ]:
adata.obs[adata.obs.predicted_doublet]

# 标准化

我们正在应用中位计数深度归一化，并采用 log1p 变换（又称 log1PF）。

In [ ]:
# Saving count data
adata.layers["counts"] = adata.X.copy()

# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
adata

# 特征选择

In [ ]:
下一步，我们希望降低数据集的维度，只包含最具信息量的基因。这一步通常被称为特征选择。

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, batch_key="sample")
sc.pl.highly_variable_genes(adata)

# 降维

通过运行主成分分析（PCA）降低数据的维度，该分析揭示了主要的变异轴并对数据进行去噪。

In [ ]:
sc.tl.pca(adata)

让我们检查单个 PC 对数据总方差的贡献。这给了我们关于计算细胞邻域关系时应考虑多少个 PC（如聚类函数 leiden（） 或 tsne（） 的信息。

根据我们的经验，高估主成分数量似乎没有显著的负面影响。

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=50, log=True)

你还可以绘制主成分图，看看是否存在任何潜在的不受欢迎特征（如批处理、质量控制指标）导致数据集显著变异。在这种情况下，没有什么特别严重的问题，但探索一下是个好主意。

In [ ]:
sc.pl.pca(
    adata,
    color=["sample", "sample", "pct_counts_mt", "pct_counts_mt"],
    dimensions=[(0, 1), (2, 3), (0, 1), (2, 3)],
    ncols=2,
    size=2,
)

# 最邻近图的构造和可视化

让我们用数据矩阵的 PCA 表示计算单元格的邻域图。

In [ ]:
sc.pp.neighbors(adata)

该图随后可以嵌入二维空间，使用 UMAP [McInnes 等，2018] 进行可视化：

In [ ]:
sc.tl.umap(adata)

我们现在可以根据样本可视化 UMAP。

In [ ]:
sc.pl.umap(
    adata,
    color="sample",
    # Setting a smaller point size to get prevent overlap
    size=2,
)

尽管本教程中涉及两个不同的样本，但我们只观察到轻微的批次效应，可以继续对数据进行聚类和注释。

如果你在 UMAP 中检查批次效应，可以对样品进行积分并进行批次校正/整合。我们建议你试试 scanorama 和 scvi-tools 来做批次整合。

# 聚类

与 Seurat 及许多其他框架一样，我们推荐 Leiden 图聚类方法（基于模块化优化的社区检测）[Traag 等 ，2019]。 注意，莱顿聚类直接聚类了胞子的邻域图，我们在上一节已经计算过。

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster,
# especially for larger datasets
sc.tl.leiden(adata, flavor="igraph", n_iterations=2)

In [ ]:
sc.pl.umap(adata, color=["leiden"])

# 重新评估质量控制和细胞过滤

如前所述，我们将通过使用 UMAP 可视化不同的质量控制指标来重新评估过滤策略。

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden", "predicted_doublet", "doublet_score"],
    # increase horizontal space between panels
    wspace=0.5,
    size=3,
)

# 手动细胞类型注释

In [ ]:
for res in [0.02, 0.5, 2.0]:
    sc.tl.leiden(adata, key_added=f"leiden_res_{res:4.2f}", resolution=res, flavor="igraph")

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden_res_0.02", "leiden_res_0.50", "leiden_res_2.00"],
    legend_loc="on data",
)

虽然 UMAP 不应被过度解读，但我们已经可以看到，在最高分辨率下我们的数据被过度聚类，而最低分辨率则很可能将属于不同单元格的单元格分组。

# 标记基因集

In [ ]:
marker_genes = {
    "CD14+ Mono": ["FCN1", "CD14"],
    "CD16+ Mono": ["TCF7L2", "FCGR3A", "LYN"],
    # Note: DMXL2 should be negative
    "cDC2": ["CST3", "COTL1", "LYZ", "DMXL2", "CLEC10A", "FCER1A"],
    "Erythroblast": ["MKI67", "HBA1", "HBB"],
    # Note HBM and GYPA are negative markers
    "Proerythroblast": ["CDK6", "SYNGR1", "HBM", "GYPA"],
    "NK": ["GNLY", "NKG7", "CD247", "FCER1G", "TYROBP", "KLRG1", "FCGR3A"],
    "ILC": ["ID2", "PLCG2", "GNLY", "SYNE1"],
    "Naive CD20+ B": ["MS4A1", "IL4R", "IGHD", "FCRL1", "IGHM"],
    # Note IGHD and IGHM are negative markers
    "B cells": [
        "MS4A1",
        "ITGB1",
        "COL4A4",
        "PRDM1",
        "IRF4",
        "PAX5",
        "BCL11A",
        "BLK",
        "IGHD",
        "IGHM",
    ],
    "Plasma cells": ["MZB1", "HSP90B1", "FNDC3B", "PRDM1", "IGKC", "JCHAIN"],
    # Note PAX5 is a negative marker
    "Plasmablast": ["XBP1", "PRDM1", "PAX5"],
    "CD4+ T": ["CD4", "IL7R", "TRBC2"],
    "CD8+ T": ["CD8A", "CD8B", "GZMK", "GZMA", "CCL5", "GZMB", "GZMH", "GZMA"],
    "T naive": ["LEF1", "CCR7", "TCF7"],
    "pDC": ["GZMB", "IL3RA", "COBLL1", "TCF4"],
}

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby="leiden_res_0.02", standard_scale="var")

In [ ]:
adata.obs["cell_type_lvl1"] 

In [ ]:
adata.obs["cell_type_lvl1"] = adata.obs["leiden_res_0.02"].map(
    {
        "0": "Lymphocytes",
        "1": "Monocytes",
        "2": "Erythroid",
        "3": "B Cells",
    }
)

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby="leiden_res_0.50", standard_scale="var")

这似乎是一个适合区分我们数据中大多数不同细胞类型的分辨率。因此，我们尝试用上面的点图和我们簇的 UMAP 手动标注这些区域。理想情况下，还应专门研究每个簇，并在必要时尝试将其子簇。

# 差异表达基因作为标记

此外，还可以计算每个簇的标记基因，然后查找是否能将这些标记基因与已知的生物特征（如细胞类型和/或状态）联系起来。这通常通过简单的统计检验，如 Wilcoxon 检验和 t 检验，对每个簇进行对比其他簇进行。

In [ ]:
# Obtain cluster-specific differentially expressed genes
sc.tl.rank_genes_groups(adata, groupby="leiden_res_0.50", method="wilcoxon")

然后我们可以在点图上可视化前5个差异表达基因。

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, groupby="leiden_res_0.50", standard_scale="var", n_genes=5)

为了创建自己的图，或采用更自动化的方法，可以通过以下方式提取差异表达的基因，以方便的格式提取。

In [ ]:
sc.get.rank_genes_groups_df(adata, group="7").head(5)

In [ ]:
dc_cluster_genes = sc.get.rank_genes_groups_df(adata, group="7").head(5)["names"]
sc.pl.umap(
    adata,
    color=[*dc_cluster_genes, "leiden_res_0.50"],
    legend_loc="on data",
    frameon=False,
    ncols=3,
)